# WML-A Model Deployment via CLI
Offical examples can be found here: https://wmla-console-cpd-wmla.apps.cpd.mskcc.org/ui/#/cliTools

In [29]:
%env DIR=/userfs/deployment-tutorial
%env HOST=wmla-console-cpd-wmla.apps.cpd.mskcc.org
%env REST_SERVER=wmla-console-cpd-wmla.apps.cpd.mskcc.org/dlim/v1/

%env dlim=../wmla-utils/dlicmd.py

env: DIR=/userfs/deployment-tutorial
env: HOST=wmla-console-cpd-wmla.apps.cpd.mskcc.org
env: REST_SERVER=wmla-console-cpd-wmla.apps.cpd.mskcc.org/dlim/v1/
env: dlim=../wmla-utils/dlicmd.py


In [21]:
!chmod +x $dlim

## Stage and deploy model
Model needs to be staged to the WML assets space and then deployed to WMLA.  The following command needs to be run in a terminal session

In [22]:
%env DIR_submission=/userfs/deployment-tutorial/deployment_submission
%env file_kernel=kernel.py

!rm -rf $DIR_submission
!mkdir -p $DIR_submission

!cp /userfs/training-tutorial/cifar-visdom/model/model.pt $DIR_submission
!cp $DIR/kernel.py $DIR_submission
!cp $DIR/model.json $DIR_submission
!cp $DIR/README.md $DIR_submission

env: DIR_submission=/userfs/deployment-tutorial/deployment_submission
env: file_kernel=kernel.py


In [30]:
!$dlim model deploy -p $DIR_submission --rest-server $REST_SERVER --rest-port -1 --jwt-token $USER_ACCESS_TOKEN

Traceback (most recent call last):
  File "../wmla-utils/dlicmd.py", line 2860, in <module>
    sys.exit(main())
  File "../wmla-utils/dlicmd.py", line 2779, in main
    get_info_from_dli_rest()
  File "../wmla-utils/dlicmd.py", line 878, in get_info_from_dli_rest
    flags.dli_rest_url = "https://" + flags.rest_host + get_dlirest_port() + DLI_REST
TypeError: can only concatenate str (not "NoneType") to str


In [ ]:
!python cli_mlops.py prepare stage --path-yml deployment_metadata_cli_example.yml --path-model /mnts/AdditionalDeepLIIFVolume/deepliif-ws-output/Test_Model_wendy_ws_serialized3 --path-dependency /userfs/wmla-deployment/edi-deployment-dirs/deepliif-base -f

In [ ]:
!python cli_mlops.py deploy stop --name deepliif-test

In [ ]:
!python cli_mlops.py deploy delete --name deepliif-test

In [ ]:
!python cli_mlops.py deploy create --model-asset-id 8ec4ed52-a390-4fa1-9114-a48ffd437bc8  --cpd-username *** --cpd-api-key *** --kernel-filename kernel.py --name deepliif-test --dlim-path /home/wsuser/bin --custom-arg CPD_USERNAME=*** --custom-arg CPD_API_KEY=*** --custom-arg VOLUME_DISPLAY_NAME=AdditionalDeepLIIFVolume

## Image on volume (recommended for large images)

In [ ]:
headers = {'Authorization': f'Bearer {os.getenv("USER_ACCESS_TOKEN")}'}
data = {
        'img_path_on_pvc':'edi_deployments/sample_input_images/22_2_real_A.png',
        'save_path_on_pvc':'edi_inference/output_images',
        'tile_size': 512,
        'images_to_return': 'seg_masks',
        }

In [ ]:
r = requests.post(DEPLOYMENT_URL, headers=headers,
                  json = data, verify = False)
r.status_code

In [ ]:
r.json()

## Send and return serialized image in request

In [ ]:
local_image = Image.open("/mnts/DeepLIIFData/edi_deployments/sample_input_images/22_2_real_A.png")

In [ ]:
headers = {'Authorization': f'Bearer {os.getenv("USER_ACCESS_TOKEN")}'}
data = {
        'local_input_image': edi.serialize_image(local_image),
        'tile_size': 512,
        'images_to_return': 'all',
        }

In [ ]:
r2 = requests.post(DEPLOYMENT_URL, headers=headers,
                  json = data, verify = False)
r2.status_code

In [ ]:
output = r2.json()

In [ ]:
output['images'].keys()

In [ ]:
output['msg']

In [ ]:
Image.open(edi.deserialize_image(output['images']['local_SegOverlaid.png']))